<a href="https://colab.research.google.com/github/sabarishraja/Transformer-pirate-style-generator/blob/main/Another_version_of_pirate_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U spacy==3.7.4 textacy==0.13.0 spacy-lookups-data
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.6/321.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.

In [ ]:
import os, re, json, csv, textwrap, requests
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex
from spacy.symbols import ORTH
import textacy
import textacy.preprocessing as tprep
from textacy.extract import ngrams
from collections import Counter
import numpy as np
import pandas as pd

In [ ]:
BOOKS = [
    ("Treasure Island (Stevenson)", "https://www.gutenberg.org/cache/epub/120/pg120.txt"),
    ("Captain Blood (Sabatini)", "https://www.gutenberg.org/ebooks/1965.txt.utf-8"),
    ("Captain Singleton (Defoe)", "https://www.gutenberg.org/ebooks/6422.txt.utf-8"),
    ("General History of the Pyrates (Johnson/Defoe)", "https://www.gutenberg.org/ebooks/40580.txt.utf-8"),
    ("Howard Pyle's Book of Pirates", "https://www.gutenberg.org/ebooks/26862.txt.utf-8"),
]
print("Books loaded:", len(BOOKS))


Books loaded: 5


# Strip to the start content of book

In [ ]:
def fetch_text(url):
    r = requests.get(url, timeout=60); r.raise_for_status(); return r.text

def strip_gutenberg(txt):
    s = re.search(r"\*\*\* START OF(.*)\*\*\*", txt)
    e = re.search(r"\*\*\* END OF(.*)\*\*\*", txt)
    if s and e and s.end() < e.start():
        txt = txt[s.end():e.start()]
    return txt.replace("\r\n","\n").strip()

def light_preprocess(s):
    s = tprep.normalize.unicode(s, form="NFKC")
    s = tprep.normalize.quotation_marks(s)
    s = tprep.normalize.hyphenated_words(s)
    s = tprep.normalize.whitespace(s)
    return s.strip()

# TEST
raw = fetch_text(BOOKS[3][1])
core = strip_gutenberg(raw)
clean = light_preprocess(core)
print("Sample clean excerpt:\n", clean[:300])
print("Chars:", len(clean))


# spaCy pipeline with "piratey" tokenizer

In [ ]:
def build_nlp():
    nlp = spacy.load("en_core_web_sm", exclude=["ner"])
    if "sentencizer" not in nlp.pipe_names and "parser" not in nlp.pipe_names:
        nlp.add_pipe("sentencizer")

    APOST = {"'", "’"}
    infixes = [i for i in nlp.Defaults.infixes if not any(ch in i for ch in APOST)]
    infix_re = compile_infix_regex(infixes)

    def tok(nlp_):
        return Tokenizer(
            nlp_.vocab,
            rules=nlp_.Defaults.tokenizer_exceptions,
            prefix_search=nlp_.tokenizer.prefix_search,
            suffix_search=nlp_.tokenizer.suffix_search,
            infix_finditer=infix_re.finditer,
            token_match=nlp_.tokenizer.token_match,
            url_match=nlp_.tokenizer.url_match,
        )
    nlp.tokenizer = tok(nlp)
    for sc in ["Cap'n","Cap’n","'em","’em","d'ye","d’ye","o'","o’","aye-aye"]:
        nlp.tokenizer.add_special_case(sc, [{ORTH: sc}])
    return nlp

# TEST
nlp = build_nlp()
test = "Cap’n Flint’s crew—d’ye hear ’em? o’ rum, aye-aye! Ahoy, matey!"
print([t.text for t in nlp.tokenizer(test)])


['Cap’n', 'Flint', '’s', 'crew', '—', 'd’ye', 'hear', '’em', '?', 'o’', 'rum', ',', 'aye-aye', '!', 'Ahoy', ',', 'matey', '!']


# Token View

In [ ]:
from spacy.lang.en import STOP_WORDS as EN_STOP

pirate_keep = {"aye","yo-ho-ho","matey","cap’n","cap'n","rum","ahoy","avast","aye-aye","blackbeard","flint","silver"}
custom_stop = set(EN_STOP) - pirate_keep

def token_view(doc, lowercase=True, remove_stop=True, remove_punct=True, remove_num=True, use_lemma=True):
    out = []
    for t in doc:
        if t.is_space: continue
        if remove_punct and t.is_punct: continue
        if remove_num and t.like_num: continue
        s = t.lemma_ if use_lemma else t.text
        s = s.lower() if lowercase else s
        if remove_stop and s in custom_stop: continue
        s = s.strip()
        if s: out.append(s)
    return out

def stats_from_doc(doc):
    toks = token_view(doc)
    vocab = Counter(toks)
    sents = [s for s in doc.sents if s.text.strip()]
    avg_len = np.mean([sum(1 for t in s if not (t.is_space or t.is_punct)) for s in sents]) if sents else 0.0
    return {
        "sentences": len(sents),
        "tokens": len(toks),
        "vocab": len(vocab),
        "avg_sent_len_tokens": float(avg_len),
        "top_tokens_15": vocab.most_common(15),
    }

# TEST
doc = nlp(clean)
print(stats_from_doc(doc))

{'sentences': 5172, 'tokens': 52035, 'vocab': 7681, 'avg_sent_len_tokens': 25.22505800464037, 'top_tokens_15': [('ship', 578), ('captain', 490), ('pyrates', 423), ('come', 386), ('man', 367), ('men', 365), ('time', 325), ('sloop', 310), ('great', 276), ('island', 256), ('find', 250), ('board', 248), ('day', 240), ('good', 233), ('pyrate', 208)]}


# Piratey Term finder

In [ ]:
pirate_patterns = [
    r"\baye\b", r"\bmatey\b", r"\byo-?ho-?ho\b", r"cap[’']n",
    r"d[’']ye", r"[’']em", r"\bahoy\b", r"\bavast\b", r"\brum\b",
]

def pirate_counts_and_kwic(text, patterns=pirate_patterns, width=35, max_hits=3):
    counts = {pat: len(re.findall(pat, text, flags=re.IGNORECASE)) for pat in patterns}
    snips = {}
    for pat in patterns:
        lines = []
        for m in re.finditer(pat, text, flags=re.IGNORECASE):
            start = max(0, m.start()-width)
            end   = min(len(text), m.end()+width)
            lines.append(text[start:end].replace("\n"," "))
            if len(lines) >= max_hits: break
        if lines:
            snips[pat] = lines
    return counts, snips

# TEST
counts, snips = pirate_counts_and_kwic(clean)
print("Pirate counts (nonzero):", {k:v for k,v in counts.items() if v>0})
for pat, lines in snips.items():
    print(f"\nKWIC {pat}:")
    for ln in lines: print("  …" + ln + "…")

Pirate counts (nonzero): {"d[’']ye": 1, "[’']em": 7, '\\brum\\b': 17}

KWIC d[’']ye:
  …ip will hear some Reason. _Judge._ D'ye hear how the Scoundrel prates?--Wh…

KWIC [’']em:
  …ile, till I bring other Company to 'em. In the Month of _November_, 1716,…
  … for we never heard what became of 'em afterwards: Captain _Hume_ release…
  …ding their Superiority, we engaged 'em both about three Hours, during whi…

KWIC \brum\b:
  … and about four hundred Gallons of Rum, besides his Provisions and Stores…
  … enough, and made them Presents of Rum and Sugar, in Recompence of what h…
  … with his own Hand.-- _Such a Day, Rum all out:--Our Company somewhat sob…


In [ ]:
def summarize_book(title, url, nlp):
    raw   = fetch_text(url)
    core  = strip_gutenberg(raw)
    clean = light_preprocess(core)
    doc   = nlp(clean)
    stats = stats_from_doc(doc)

    # top n-grams
    bi = Counter([sp.text.lower() for sp in ngrams(doc, n=2, filter_stops=True, filter_punct=True, filter_nums=True)])
    tri = Counter([sp.text.lower() for sp in ngrams(doc, n=3, filter_stops=True, filter_punct=True, filter_nums=True)])

    # piratey terms
    counts, snips = pirate_counts_and_kwic(clean)

    return {
        "title": title,
        "chars": len(clean),
        "stats": stats,
        "bigrams_top10": bi.most_common(10),
        "trigrams_top10": tri.most_common(10),
        "pirate_counts": counts,
        "pirate_kwic": snips,
    }

# TEST on Treasure Island only
one = summarize_book(*BOOKS[0], nlp)
print(one["title"], "chars:", one["chars"])
print("stats:", one["stats"])
print("bigrams:", one["bigrams_top10"][:5])
print("pirate (nonzero):", {k:v for k,v in one["pirate_counts"].items() if v>0})

Treasure Island (Stevenson) chars: 361727
stats: {'sentences': 3660, 'tokens': 25841, 'vocab': 4382, 'avg_sent_len_tokens': 19.223224043715845, 'top_tokens_15': [('man', 367), ('captain', 236), ('silver', 223), ('like', 214), ('come', 211), ('hand', 197), ('doctor', 175), ('good', 151), ('know', 140), ('time', 139), ('cry', 137), ('look', 135), ('ship', 134), ('long', 127), ('think', 127)]}
bigrams: [('dr. livesey', 44), ('ben gunn', 38), ('long john', 37), ('captain smollett', 29), ('said silver', 27)]
pirate (nonzero): {'\\baye\\b': 27, '\\bmatey\\b': 4, '\\byo-?ho-?ho\\b': 8, "cap[’']n": 48, "[’']em": 31, '\\bahoy\\b': 1, '\\bavast\\b': 2, '\\brum\\b': 57}


In [ ]:
chars = sorted(list(set(clean)))
vocab_size = len(chars)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}

def encode(s: str):
    return [stoi[c] for c in s]

def decode(ixs):
    return "".join(itos[i] for i in ixs)

print("Vocab size:", vocab_size)
print("Sample mapping:", list(stoi.items())[:15])
print("Endcoding a text(Cap'n): ", encode("Cap'n"))
print("Encode/Decode test:", decode(encode("Cap'n")) )

Vocab size: 88
Sample mapping: [('\n', 0), (' ', 1), ('!', 2), ('&', 3), ("'", 4), ('(', 5), (')', 6), ('*', 7), ('+', 8), (',', 9), ('-', 10), ('.', 11), ('0', 12), ('1', 13), ('2', 14)]
Endcoding a text(Cap'n):  [27, 54, 69, 4, 67]
Encode/Decode test: Cap'n


# Starting Bilanguage model

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math, random, time

In [ ]:
#Building a single corpus for all 5 books
def build_corpus_from_books(books):
    pieces = []
    for title, url in books:
        raw   = fetch_text(url)
        core  = strip_gutenberg(raw)
        clean = light_preprocess(core)   # <-- your cleaner; keeps punctuation
        pieces.append(clean)
    return "\n\n".join(pieces)

corpus = build_corpus_from_books(BOOKS)
print("Total chars in corpus:", len(corpus))


Total chars in corpus: 2806865


In [ ]:
#Rebuilding character vocab
chars = sorted(list(set(corpus)))
vocab_size = len(chars)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for ch,i in stoi.items()}

def encode(s: str):
  return torch.tensor([stoi[c] for c in s], dtype = torch.long)

def decode(ixs):
    return "".join(itos[int(i)] for i in ixs)

data = encode(corpus)
print("Vocab size:", vocab_size, "| data shape:", tuple(data.shape))

Vocab size: 93 | data shape: (2806865,)


# Train

In [ ]:
torch.manual_seed(1337)
split = int(0.9 * len(data))
train_data = data[:split]
val_data   = data[split:]

In [ ]:
block_size = 128
batch_size = 32

def get_batch(split_name):
    src = train_data if split_name == "train" else val_data
    ix = torch.randint(0, len(src) - block_size - 1, (batch_size,))
    x = torch.stack([src[i:i+block_size]     for i in ix])
    y = torch.stack([src[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, V)
        loss = None
        if targets is not None:
            B, T, V = logits.shape
            logits = logits.view(B*T, V)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    @torch.no_grad()
    def generate(self, start_idx, max_new_tokens):
        idx = start_idx
        for _ in range(max_new_tokens):
            logits, _ = self(idx)         # (B, T, V)
            logits = logits[:, -1, :]     # (B, V)
            probs = F.softmax(logits, dim=-1)
            next_tok = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_tok], dim=1)
        return idx

In [ ]:
model = BigramLanguageModel(vocab_size)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-3, betas=(0.9, 0.95), weight_decay=1e-4)

In [ ]:
@torch.no_grad()
def estimate_loss(eval_iters=200):
    model.eval()
    losses = {}
    for split_name in ["train", "val"]:
        vals = []
        for _ in range(eval_iters):
            X, Y = get_batch(split_name)
            _, loss = model(X, Y)
            vals.append(loss.item())
        losses[split_name] = sum(vals) / len(vals)
    model.train()
    return losses


In [ ]:
max_iters = 3000
eval_interval = 500
grad_clip = 1.0

t0 = time.time()
for step in range(max_iters + 1):
    if step % eval_interval == 0:
        losses = estimate_loss(eval_iters=200)
        print(f"[{step:5d}] train {losses['train']:.4f} | val {losses['val']:.4f} | {time.time()-t0:.1f}s")

    X, Y = get_batch("train")
    _, loss = model(X, Y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    optimizer.step()

print("Training complete.")

[    0] train 5.0195 | val 5.0253 | 2.1s
[  500] train 3.2599 | val 3.2606 | 10.4s
[ 1000] train 2.5863 | val 2.5896 | 21.1s
[ 1500] train 2.4747 | val 2.4773 | 29.8s
[ 2000] train 2.4608 | val 2.4639 | 37.4s
[ 2500] train 2.4610 | val 2.4646 | 44.3s
[ 3000] train 2.4561 | val 2.4621 | 51.8s
Training complete.


In [ ]:
model.eval()
with torch.no_grad():
    start = torch.randint(vocab_size, (1, 1))
    out = model.generate(start, max_new_tokens=800)[0].tolist()
    sample = decode(out)

print("\n=== SAMPLE OUTPUT ===\n")
print(sample[:1200])


=== SAMPLE OUTPUT ===

́usilee. op theed mecharde, asied. gicof ss aitan od?"y liswie t ofambed ondeas_Cou[I he he ordld atowethed tho me senoind in, sowin theys herof asuze rulil.
_."Anen t merend oure wamen befo
oum cre 2, Yo' bullad owowarmat angrthes, e hime a fot Band, hare Jandstosur g pind st Cond I se t, paner Apprs. the Shensod the Credmerend, Pe
Dout he arm Cæthm werschaindanonde fird llanvend fr.
"
I otomort pe t An, f ajundasom y, whed cads Ang Inged purs thefot rned nhe hels
Afon he Enghe ch l cthe tsinth
Fl ney t t s cher
manowhice bofit_Dunginomshilund weand--I cthasth ounon he
Chivecquck Feva o t w
waitsse he wed urad; altaspinrunuerin wand, d his t s onghige henove athe."Me t r f
n here wor ie he-pteldrdrse no aifr alll. Tustr Cat acy d tes allinsst; foulan s whicoo ger s ak h s, as,"
t oas! ng f


# Building a Transformer model

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size, n_embd, context_size, dropout=0.1):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # causal mask: lower-triangular (no looking ahead)
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size, dtype=torch.bool)))
        self.scale = head_size ** -0.5    # scaled dot-product (more stable)
        self.drop  = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)                      # (B,T,hd)
        q = self.query(x)                    # (B,T,hd)
        # attention scores
        wei = (q @ k.transpose(-2, -1)) * self.scale   # (B,T,T)
        wei = wei.masked_fill(~self.tril[:T, :T], float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.drop(wei)                 # dropout on attention weights

        v = self.value(x)                    # (B,T,hd)
        out = wei @ v                        # (B,T,hd)
        return out


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, n_embd, context_size, dropout=0.1):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, context_size, dropout) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd, bias=False) # mix heads
        self.drop  = nn.Dropout(dropout)

    def forward(self, x):
        # concat along channel dim, then project back
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # (B,T,n_embd)
        out = self.drop(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ linear -> ReLU -> linear (small/fast) """
    def __init__(self, n_embd, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x): return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head, context_size, dropout=0.1):
        super().__init__()
        head_size = n_embd // n_head
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa  = MultiHeadAttention(n_head, head_size, n_embd, context_size, dropout)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffw = FeedFoward(n_embd, dropout)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  # residual 1
        x = x + self.ffw(self.ln2(x)) # residual 2
        return x

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size, n_embd=192, context_size=64, n_head=3, n_layer=3, dropout=0.1):
        super().__init__()
        assert n_embd % n_head == 0, "n_embd must be divisible by n_head"
        self.context_size = context_size

        # token + position embeddings
        self.token_embedding_table    = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(context_size, n_embd)

        # stack of transformer blocks
        self.blocks = nn.Sequential(*[
            Block(n_embd, n_head=n_head, context_size=context_size, dropout=dropout)
            for _ in range(n_layer)
        ])

        self.ln_f   = nn.LayerNorm(n_embd)     # final norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # positions 0..T-1
        pos = torch.arange(T, device=idx.device)
        # embed tokens + positions
        x = self.token_embedding_table(idx) + self.position_embedding_table(pos)  # (B,T,C)
        # transformer stack
        x = self.blocks(x)
        x = self.ln_f(x)
        # logits over vocab
        logits = self.lm_head(x)  # (B,T,V)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens=200):
        # autoregressive sampling (keep last context_size)
        for _ in range(max_new_tokens):
            x = idx[:, -self.context_size:]
            logits, _ = self(x)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

In [ ]:
@torch.no_grad()
def quick_eval(model, iters=10):
    model.eval()
    losses = []
    for _ in range(iters):
        X, Y = get_batch('val')
        _, loss = model(X, Y)
        losses.append(loss.item())
    model.train()
    return sum(losses)/len(losses)

In [ ]:
def train_fast(model,
               lr=5e-4,
               max_iters=1200,
               eval_every=300,
               mini_ping=100,
               sample_tokens=500):
    print("🚀 training (tiny transformer)")
    opt = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9,0.95), weight_decay=1e-2)

    t0 = time.time()
    for step in range(max_iters + 1):
        if step % mini_ping == 0 and step % eval_every != 0:
            print(f"  ...step {step}")

        if step % eval_every == 0:
            val = quick_eval(model, iters=10)
            print(f"[{step:4d}] val {val:.3f} | {time.time()-t0:.1f}s")

        X, Y = get_batch('train')
        _, loss = model(X, Y)
        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()

    print("✅ done. sampling…")
    with torch.no_grad():
        start = torch.randint(vocab_size, (1,1))
        out = model.generate(start, max_new_tokens=sample_tokens)[0].tolist()
    print("\n=== TRANSFORMER SAMPLE OUTPUT ===\n")
    print(decode(out))

In [ ]:
context_size = 64
block_size = context_size
m = BigramLanguageModel(vocab_size, n_embd=192, context_size=context_size, n_head=3, n_layer=3, dropout=0.1)
train_fast(m, max_iters=1200, eval_every=300, mini_ping=100, sample_tokens=600)

🚀 training (tiny transformer)
[   0] val 4.642 | 1.4s
  ...step 100
  ...step 200
[ 300] val 2.253 | 133.4s
  ...step 400
  ...step 500
[ 600] val 2.028 | 264.6s
  ...step 700
  ...step 800
[ 900] val 1.936 | 393.2s
  ...step 1000
  ...step 1100
[1200] val 1.870 | 523.5s
✅ done. sampling…

=== TRANSFORMER SAMPLE OUTPUT ===

y the three to go them of the sucalfound, it mark geney been-ous to
thin ander let the men to cond there, hiss dind, and were conquast
andsted jectiction, all bult the prond_'d mantegen _Pyle_; Gunl_ Lived dirkt, teen Mark ship the
ment and, and lif the To Mitusted bood. Wors, is abledely. WIll seme
Sporlood one, with, the then leations prouse of a good. Mass, hart's clobely, threase agoon bleds
erenges, Cat Willinly-you evelys thing doner throught, notshed; 172
"Prysiails a ling a Peor. Ol the shiccoof therd, that of the Ben of the
siefar and thech, I share therelverain.
Wor the Gungr. It aido
